Verinin okunması ve eksik veri kontrolü

In [1]:
import pandas as pd
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C


df = pd.read_csv('Life Expectancy Data.csv')

missing_data = df[df.isnull().any(axis=1)]

print(missing_data)


       Country  Year      Status  Life expectancy   Adult Mortality  \
32     Algeria  2015  Developing              75.6             19.0   
44     Algeria  2003  Developing              71.7            146.0   
45     Algeria  2002  Developing              71.6            145.0   
46     Algeria  2001  Developing              71.4            145.0   
47     Algeria  2000  Developing              71.3            145.0   
...        ...   ...         ...               ...              ...   
2918    Zambia  2003  Developing              46.4             64.0   
2919    Zambia  2002  Developing              45.5             69.0   
2920    Zambia  2001  Developing              44.6            611.0   
2921    Zambia  2000  Developing              43.8            614.0   
2922  Zimbabwe  2015  Developing              67.0            336.0   

      infant deaths  Alcohol  percentage expenditure  Hepatitis B  Measles   \
32               21      NaN                0.000000         95.0   

In [2]:
missing_counts = df.isnull().sum()

print("Her sütundaki eksik veri sayıları:")
print(missing_counts)


Her sütundaki eksik veri sayıları:
Country                              0
Year                                 0
Status                               0
Life expectancy                     10
Adult Mortality                     10
infant deaths                        0
Alcohol                            194
percentage expenditure               0
Hepatitis B                        553
Measles                              0
 BMI                                34
under-five deaths                    0
Polio                               19
Total expenditure                  226
Diphtheria                          19
 HIV/AIDS                            0
GDP                                448
Population                         652
 thinness  1-19 years               34
 thinness 5-9 years                 34
Income composition of resources    167
Schooling                          163
dtype: int64


Girdi ve çıktıların ayrılaması. Girdide bulunan eksiklerin giderilmesi.

In [3]:
from sklearn.impute import KNNImputer  


y = df['Life expectancy '] 
X = df.drop(columns=['Life expectancy ']).select_dtypes(include=['number']) 

imputed_X = X.copy()

impute_knn = KNNImputer(n_neighbors=10)
imputed_data = impute_knn.fit_transform(X)

imputed_df = pd.DataFrame(imputed_data)

imputed_X = imputed_df

print(imputed_X.isna().sum())




0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    0
18    0
dtype: int64


GaussianProcess ile çıktıdaki eksiklerin giderilmesi.

In [4]:
missing_y_idx = y[y.isnull()].index


not_missing_y = y.dropna()
not_missing_X = imputed_X.loc[not_missing_y.index]

kernel = C(1.0, (1e-4, 1e1)) * RBF(1.0, (1e-4, 1e1))  
gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10, random_state=42)


gp.fit(not_missing_X, not_missing_y)


imputed_y = gp.predict(imputed_X.loc[missing_y_idx])

y_imputed = y.copy()
y_imputed[missing_y_idx] = imputed_y

print(y_imputed.isna().sum())

c:\Users\New\anaconda3\Lib\site-packages\sklearn\gaussian_process\kernels.py:430: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 10.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
c:\Users\New\anaconda3\Lib\site-packages\sklearn\gaussian_process\kernels.py:430: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified upper bound 10.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(


0
